# Simulated KB Mirror Demo

This notebook introduces the use of Blop to tune a KB mirror pair to optimize the quality of a simulated beam read by a detector.

Blop uses [Ax](https://ax.dev) as its optimization and experiment tracking backend.

Ax provides:
- Experiment tracking
- Analysis & visualization
- Bayesian optimization (through [BoTorch](https://botorch.org/))

Blop provides:
- Native integration with [Bluesky & its ecosystem](https://blueskyproject.io)
- Specialized kernels and methods common to beamline optimization problems

These features make it simple to optimize your beamline using both Bluesky & Ax.

## Preparing a test environment

Here we prepare the `RunEngine` and data service `Databroker`.

In [ ]:
from blop.utils import prepare_re_env  # noqa

%run -i $prepare_re_env.__file__ --db-type=temp
# Disable the standard plots that Bluesky creates
bec.disable_plots()

import plotly.io as pio
pio.renderers.default = "notebook"

## Simulated beamline with KB mirror pair

Here we describe an analytical simulated beamline with a [KB mirror](https://en.wikipedia.org/wiki/Kirkpatrick%E2%80%93Baez_mirror) pair. This is implemented as an [Ophyd](https://blueskyproject.io/ophyd/) device for ease-of-use with Bluesky.

In [ ]:
from blop.sim import Beamline

beamline = Beamline(name="bl")
beamline.det.noise.put(False)

## Create a Blop-Ax experiment

Now we can define the experiment we plan to run.

This involves setting 4 parameters that simulate motor positions controlling two KB mirrors. The objectives of the experiment are to maximize the beam intensity while minimizing the area of the beam.

In [ ]:
from blop.ax import Agent
from blop.dofs import DOF
from blop.objectives import Objective

dofs = [
    DOF(device=beamline.kbv_dsv, type="continuous", search_domain=(-5.0, 5.0)),
    DOF(device=beamline.kbv_usv, type="continuous", search_domain=(-5.0, 5.0)),
    DOF(device=beamline.kbh_dsh, type="continuous", search_domain=(-5.0, 5.0)),
    DOF(device=beamline.kbh_ush, type="continuous", search_domain=(-5.0, 5.0)),
]

objectives = [
    Objective(name="bl_det_sum", target="max"),
    Objective(name="bl_det_wid_x", target="min", transform="log"),
    Objective(name="bl_det_wid_y", target="min", transform="log"),
]

agent = Agent(
    readables=[beamline.det],
    dofs=dofs,
    objectives=objectives,
    db=db,
)
agent.configure_experiment(name="test_ax_agent", description="Test the AxAgent")

## Optimization

With all of our experimental setup done, we can optimize the DOFs to satisfy our objectives.

For this example, Ax will optimize the 4 motor positions to produce the greatest intensity beam with the smallest beam width and height (smallest area). It does this by first running a couple of trials which are random samples, then the remainder using Bayesian optimization through BoTorch.

In [ ]:
RE(agent.learn(iterations=25, n=1))

## Analyze Results

We can start by summarizing each step of the optimization procedure and whether trials were successful or not.

In [ ]:
agent.summarize()

### Plotting

We also can plot slices of the parameter space with respect to our objectives.

In [ ]:
from ax.analysis import SlicePlot

_ = agent.compute_analyses(analyses=[SlicePlot("bl_kbv_dsv", "bl_det_sum")])

In [ ]:
_ = agent.compute_analyses(analyses=[SlicePlot("bl_kbv_dsv", "bl_det_wid_x")])

### More comprehensive analyses

Ax provides many analysis tools that can help understand optimization results.

In [ ]:
from ax.analysis import TopSurfacesAnalysis

_ = agent.compute_analyses(analyses=[TopSurfacesAnalysis("bl_det_sum")])

### Visualizing the optimal beam

Below we get the optimal parameters, move the motors to their optimal positions, and observe the resulting beam.

In [ ]:
optimal_parameters = next(iter(agent.client.get_pareto_frontier()))[0]
optimal_parameters

In [ ]:
from bluesky.plans import list_scan

scan_motor_params = []
for motor in [beamline.kbv_dsv, beamline.kbv_usv, beamline.kbh_dsh, beamline.kbh_ush]:
    scan_motor_params.append(motor)
    scan_motor_params.append([optimal_parameters[motor.name]])
RE(list_scan([beamline.det], *scan_motor_params))

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(db[-1].table(fill=True)["bl_det_image"].iloc[0])
plt.colorbar()
plt.show()